In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from scombi_do.app import scombi
import sys
sys.path.append('..')
from helpers import item_to_img_overlay, my_read_method, stage
from pystac import STAC_IO, read_file, Item, Catalog, CatalogType
from shapely.geometry import shape
from ipyleaflet import Map

### Stage the STAC catalogs

In [3]:
input_references = ['https://supervisor-cpe.terradue.com/catalog/acquisitions/call-790/790-KARI-KOMPSAT3-urn_ogc_def_EOP_KARI_KOMPSAT3_K3_20201126183832_45506_19161093_L1G_Aux_xml/K3_20201126183832_45506_19161093_L1G-calibrated.json',
                    'https://terradue-stac.gitlab.io/sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414.json', 
                    'https://terradue-stac.gitlab.io/sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.json',
                    'https://terradue-stac.gitlab.io/sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720.json',
                    'https://terradue-stac.gitlab.io/sentinel-s2-l2a-cogs/48/M/YT/S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429/S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429.json']

In [4]:
catalogs = stage(input_references)

2020-12-14T10:11:57 DEBUG    Starting new HTTPS connection (1): supervisor-cpe.terradue.com:443
2020-12-14T10:11:57 DEBUG    https://supervisor-cpe.terradue.com:443 "GET /catalog/acquisitions/call-790/790-KARI-KOMPSAT3-urn_ogc_def_EOP_KARI_KOMPSAT3_K3_20201126183832_45506_19161093_L1G_Aux_xml/K3_20201126183832_45506_19161093_L1G-calibrated.json HTTP/1.1" 200 None
2020-12-14T10:11:57 DEBUG    Starting new HTTPS connection (1): terradue-stac.gitlab.io:443


* <Catalog id=K3_20201126183832_45506_19161093_L1G-calibrated>
  * <Item id=K3_20201126183832_45506_19161093_L1G-calibrated>


2020-12-14T10:11:58 DEBUG    https://terradue-stac.gitlab.io:443 "GET /sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414.json HTTP/1.1" 200 17169
2020-12-14T10:11:58 DEBUG    Starting new HTTPS connection (1): terradue-stac.gitlab.io:443


* <Catalog id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>
  * <Item id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>


2020-12-14T10:11:58 DEBUG    https://terradue-stac.gitlab.io:443 "GET /sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.json HTTP/1.1" 200 17119
2020-12-14T10:11:58 DEBUG    Starting new HTTPS connection (1): terradue-stac.gitlab.io:443


* <Catalog id=S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348>
  * <Item id=S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348>


2020-12-14T10:11:59 DEBUG    https://terradue-stac.gitlab.io:443 "GET /sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720.json HTTP/1.1" 200 17116
2020-12-14T10:11:59 DEBUG    Starting new HTTPS connection (1): terradue-stac.gitlab.io:443


* <Catalog id=S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720>
  * <Item id=S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720>


2020-12-14T10:11:59 DEBUG    https://terradue-stac.gitlab.io:443 "GET /sentinel-s2-l2a-cogs/48/M/YT/S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429/S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429.json HTTP/1.1" 200 20153


* <Catalog id=S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429>
  * <Item id=S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429>


In [5]:
catalogs[0]

'/workspace/scombi-do/demo/notebook/K3_20201126183832_45506_19161093_L1G-calibrated'

### Scombi-do

#### Simple RGB combination

The reflectances are encoded with a scaling factor of 10000. 

The S-expressions transform that value from [0,10000] to [0,1] using the numpy `interp` function

S-expressions (or symbolic expressions, abbreviated as sexprs) are a notation for nested list (tree-structured) data, invented for and popularized by the programming language Lisp

`v1`, `v2` and `v3` are the numpy arrays read from the input product band

In [6]:
expressions = ['(interp v1 (asarray 0 10000) (asarray 0 1))', 
               '(interp v2 (asarray 0 10000) (asarray 0 1))',
               '(interp v3 (asarray 0 10000) (asarray 0 1))']

Define the bands for the RGB channels using the common band names

In [14]:
bands = ['red', 'green', 'blue']

Define the input local STAC catalogs

In [8]:
channel_inputs = [os.path.join(catalogs[0], 'catalog.json'),
                  os.path.join(catalogs[0], 'catalog.json'),
                  os.path.join(catalogs[0], 'catalog.json')]
channel_inputs

['/workspace/scombi-do/demo/notebook/K3_20201126183832_45506_19161093_L1G-calibrated/catalog.json',
 '/workspace/scombi-do/demo/notebook/K3_20201126183832_45506_19161093_L1G-calibrated/catalog.json',
 '/workspace/scombi-do/demo/notebook/K3_20201126183832_45506_19161093_L1G-calibrated/catalog.json']

In [9]:
color = 'Gamma RGB 3.5 Saturation 1.4 Sigmoidal RGB 15 0.35' 

In [10]:
#aoi = 'POLYGON((136.707 -35.991,136.707 -35.804,137.071 -35.804,137.071 -35.991,136.707 -35.991))'

In [15]:
params = dict()

params['channel_inputs'] = channel_inputs
params['bands'] = bands
params['s_expressions'] = expressions
params['color'] = color
#params['aoi'] = aoi


In [16]:
result = scombi(**params)

2020-12-14T10:13:15 INFO     <Item id=K3_20201126183832_45506_19161093_L1G-calibrated>
2020-12-14T10:13:15 INFO     <Item id=K3_20201126183832_45506_19161093_L1G-calibrated>
2020-12-14T10:13:15 INFO     <Item id=K3_20201126183832_45506_19161093_L1G-calibrated>
2020-12-14T10:13:16 INFO     Rescaling and COG for input assets
2020-12-14T10:13:16 INFO     Getting band red from /vsicurl/https://supervisor-cpe.terradue.com/catalog/acquisitions/call-790/790-KARI-KOMPSAT3-urn_ogc_def_EOP_KARI_KOMPSAT3_K3_20201126183832_45506_19161093_L1G_Aux_xml/red.tif


/vsicurl/https://supervisor-cpe.terradue.com/catalog/acquisitions/call-790/790-KARI-KOMPSAT3-urn_ogc_def_EOP_KARI_KOMPSAT3_K3_20201126183832_45506_19161093_L1G_Aux_xml/red.tif
open
/vsicurl/https://supervisor-cpe.terradue.com/catalog/acquisitions/call-790/790-KARI-KOMPSAT3-urn_ogc_def_EOP_KARI_KOMPSAT3_K3_20201126183832_45506_19161093_L1G_Aux_xml/green.tif
open
/vsicurl/https://supervisor-cpe.terradue.com/catalog/acquisitions/call-790/790-KARI-KOMPSAT3-urn_ogc_def_EOP_KARI_KOMPSAT3_K3_20201126183832_45506_19161093_L1G_Aux_xml/blue.tif
open
['EPSG:32617', 'EPSG:32617', 'EPSG:32617']


2020-12-14T10:13:18 INFO     Getting band green from /vsicurl/https://supervisor-cpe.terradue.com/catalog/acquisitions/call-790/790-KARI-KOMPSAT3-urn_ogc_def_EOP_KARI_KOMPSAT3_K3_20201126183832_45506_19161093_L1G_Aux_xml/green.tif
2020-12-14T10:13:20 INFO     Getting band blue from /vsicurl/https://supervisor-cpe.terradue.com/catalog/acquisitions/call-790/790-KARI-KOMPSAT3-urn_ogc_def_EOP_KARI_KOMPSAT3_K3_20201126183832_45506_19161093_L1G_Aux_xml/blue.tif
2020-12-14T10:13:22 INFO     Build VRT
2020-12-14T10:13:22 INFO     2.8
2020-12-14T10:13:22 INFO     Pimp me
2020-12-14T10:13:22 INFO     3
2020-12-14T10:13:27 INFO     Applying color operations: Gamma RGB 3.5 Saturation 1.4 Sigmoidal RGB 15 0.35
2020-12-14T10:14:08 INFO     Adding band 1 of 3
2020-12-14T10:14:09 INFO     Adding band 2 of 3
2020-12-14T10:14:09 INFO     Adding band 3 of 3
2020-12-14T10:14:15 INFO     STAC
2020-12-14T10:14:15 INFO     Done!


In [ ]:
item = next(read_file(result).get_items())

m = Map(center=(shape(item.geometry).centroid.y, 
                shape(item.geometry).centroid.x), 
                zoom=10)

image = item_to_img_overlay(item)

m.add_layer(image)
m

#### Normalized difference

In [ ]:
expressions = ['(interp (/ (- v1 v2) (+ v1 v2)) (asarray -1 1) (asarray 0 1))']

Define the bands for the RGB channels using the common band names

In [ ]:
bands = ['nir', 'red']

Define the input local STAC catalogs

In [ ]:
channel_inputs = [os.path.join(catalogs[0], 'catalog.json'), 
                  os.path.join(catalogs[0], 'catalog.json')]
channel_inputs

In [ ]:
aoi = 'POLYGON((136.707 -35.991,136.707 -35.804,137.071 -35.804,137.071 -35.991,136.707 -35.991))'

In [ ]:
params = dict()

params['channel_inputs'] = channel_inputs
params['bands'] = bands
params['s_expressions'] = expressions
params['aoi'] = aoi
params['lut'] = 'viridis'

In [ ]:
result = scombi(**params)

In [ ]:
item = next(read_file(result).get_items())

m = Map(center=(shape(item.geometry).centroid.y, 
                shape(item.geometry).centroid.x), 
                zoom=10)

image = item_to_img_overlay(item)

m.add_layer(image)
m

### Multitemporal RGB composite

In [ ]:
expressions = ['(interp v1 (asarray 0 10000) (asarray 0 1))', 
               '(interp v2 (asarray 0 10000) (asarray 0 1))',
               '(interp v3 (asarray 0 10000) (asarray 0 1))']

Define the bands for the RGB channels using the common band names

In [ ]:
bands = ['red', 'green', 'blue']

Define the input local STAC catalogs

In [ ]:
channel_inputs = [os.path.join(catalogs[0], 'catalog.json'), 
                  os.path.join(catalogs[1], 'catalog.json'),
                  os.path.join(catalogs[2], 'catalog.json')]
channel_inputs

In [ ]:
color = 'Gamma RGB 3.5 Saturation 1.4 Sigmoidal RGB 15 0.45' 

In [ ]:
aoi = 'POLYGON((136.707 -35.991,136.707 -35.804,137.071 -35.804,137.071 -35.991,136.707 -35.991))'

In [ ]:
params = dict()

params['channel_inputs'] = channel_inputs
params['bands'] = bands
params['s_expressions'] = expressions
params['color'] = color
params['aoi'] = aoi


In [ ]:
result = scombi(**params)

In [ ]:
item = next(read_file(result).get_items())

m = Map(center=(shape(item.geometry).centroid.y, 
                shape(item.geometry).centroid.x), 
                zoom=10)

image = item_to_img_overlay(item)

m.add_layer(image)
m